In [1]:
#=!pip install transformers

In [2]:
#!pip install sentence-transformers

In [4]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [6]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [7]:
import pandas as pd
citants = pd.read_json('From-ScisummNet-2019/A00-1031/citing_sentences.json')

In [8]:
citants = citants[['citance_No','clean_text']]
citants
queries = list(citants['clean_text'])
# cit_no = list(citants['citance_No'])
# query_sent = citants['raw_text']
# query_sent

In [9]:
cite_no = list(citants.citance_No)

In [10]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
tree = ET.parse("From-ScisummNet-2019/A00-1031"+"/Reference_XML/"+"A00-1031"+".xml")
root = tree.getroot()
final1=[]
final2=[]
i = 0
total = len(root)
# print(total)
for a in root:
  #  print(a.attrib)
   # print(i)
    for b in a:

        final1.append(b.text)
        if i == 0:
            final2.append("Abstract")
        if i == 1:
            final2.append("Introduction")
        elif i < total-2:
            final2.append("Experiment/Discussion")
        if i == total-2 or i == total-1:
            final2.append("Results/Conclusion")
        if i == total:
            final2.append("Acknowledgment")
    i = i+1


d={'col1':final1,'col2':final2}
rp = pd.DataFrame(data=d)

In [11]:
rp.head

<bound method NDFrame.head of                                                   col1                col2
0    Trigrams'n'Tags (TnT) is an efficient statisti...        Introduction
1    Contrary to claims found elsewhere in the lite...        Introduction
2    A recent comparison has even shown that TnT pe...        Introduction
3    We describe the basic model of TnT, the techni...        Introduction
4    Furthermore, we present evaluations on two cor...        Introduction
..                                                 ...                 ...
173  Many thanks go to Hans Uszkoreit for his suppo...  Results/Conclusion
174  Most of the work on TnT was carried out while ...  Results/Conclusion
175  Large annotated corpora are the pre-requisite ...  Results/Conclusion
176  Therefore, I would like to thank all the peopl...  Results/Conclusion
177  And, last but not least, I would like to thank...  Results/Conclusion

[178 rows x 2 columns]>

In [12]:
rp.col1

0      Trigrams'n'Tags (TnT) is an efficient statisti...
1      Contrary to claims found elsewhere in the lite...
2      A recent comparison has even shown that TnT pe...
3      We describe the basic model of TnT, the techni...
4      Furthermore, we present evaluations on two cor...
                             ...                        
173    Many thanks go to Hans Uszkoreit for his suppo...
174    Most of the work on TnT was carried out while ...
175    Large annotated corpora are the pre-requisite ...
176    Therefore, I would like to thank all the peopl...
177    And, last but not least, I would like to thank...
Name: col1, Length: 178, dtype: object

In [13]:
ann = "From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt"
with open(ann,"r") as f:
    data = f.read()

In [14]:
idx = {'Citance Number':0,'Reference Article':1,'Citing Article':2,'Citation Marker Offset':3,'Citation Marker':4,'Citation Offset':5,'Reference Offset':7}

In [15]:
gt = {}
lines = data.split('\n')
for line in lines:
    lis = line.split('|')
#     print(lis, len(lis))
    if len(lis)==11:
            cit_no = lis[0].split(':')[1]
            ref_off = lis[7].split(':')[1].strip()
            exec("ref_off="+ref_off)
            ref_off = list(map(int, ref_off))
            gt[int(cit_no)] = ref_off #

In [16]:
gt

{1: [0, 36],
 2: [44, 178],
 3: [15, 173],
 4: [0, 1],
 5: [27, 165],
 6: [27, 141],
 7: [115, 154],
 8: [24, 165],
 9: [36, 46],
 10: [2, 75],
 11: [0, 1],
 12: [18, 40],
 13: [36, 118],
 14: [24, 173],
 15: [79, 80],
 16: [0, 113],
 17: [12, 19],
 18: [0, 1],
 19: [28, 157],
 20: [36, 164]}

In [17]:
# def clean(line):
#     line = line.split("|")
#     cit_2 , ref_2 = line[6].split("Citation Text:")[1].strip() , line[8].split("Reference Text:")[1].strip()
#     cit_2 = cit_2.split(" >")[1].split("</S>")[0]
#     ref_2 = ref_2.split(" >")[1:]
#     ref_2 = [ i.split("</S>")[0] for i in ref_2]
#     ans=""
#     for i in ref_2:
#         ans=ans + cit_2 + " $$$$$ " + i +"\n"
#    # ref_2 = " ".join(ref_2)
    
#     return ans

In [18]:
# with open('./From-ScisummNet-2019/A00-1031/annotation/A00-1031.ann.txt') as f:
#     data = f.read()
# data = [ clean(i) for i in data.split("\n") if i ]

# # with open("cleaned.txt","w+") as f:
# #     f.write("\n".join(data))

In [19]:
corpus = rp.col1

In [20]:
cite_no

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [21]:
import scipy
import numpy as np

In [22]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess(example_sent):
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(example_sent.lower())

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [w for w in filtered_sentence if w.isalpha()]
    print(filtered_sentence)
    new = " " 
    a = new.join(filtered_sentence)
    return a

In [23]:
corpus

0      Trigrams'n'Tags (TnT) is an efficient statisti...
1      Contrary to claims found elsewhere in the lite...
2      A recent comparison has even shown that TnT pe...
3      We describe the basic model of TnT, the techni...
4      Furthermore, we present evaluations on two cor...
                             ...                        
173    Many thanks go to Hans Uszkoreit for his suppo...
174    Most of the work on TnT was carried out while ...
175    Large annotated corpora are the pre-requisite ...
176    Therefore, I would like to thank all the peopl...
177    And, last but not least, I would like to thank...
Name: col1, Length: 178, dtype: object

In [24]:
new_corpus = corpus.apply(lambda x: preprocess(x))

['tnt', 'efficient', 'statistical', 'tagger']
['contrary', 'claims', 'found', 'elsewhere', 'literature', 'argue', 'tagger', 'based', 'markov', 'models', 'performs', 'least', 'well', 'current', 'approaches', 'including', 'maximum', 'entropy', 'framework']
['recent', 'comparison', 'even', 'shown', 'tnt', 'performs', 'significantly', 'better', 'tested', 'corpora']
['describe', 'basic', 'model', 'tnt', 'techniques', 'used', 'smoothing', 'handling', 'unknown', 'words']
['furthermore', 'present', 'evaluations', 'two', 'corpora']
['large', 'number', 'current', 'language', 'processing', 'systems', 'use', 'tagger']
['tagger', 'assigns', 'unique', 'ambiguous', 'tag', 'token', 'input', 'passes', 'output', 'next', 'processing', 'level', 'usually', 'parser']
['furthermore', 'large', 'interest', 'tagging', 'corpus', 'annotation', 'projects', 'create', 'valuable', 'linguistic', 'resources', 'combination', 'automatic', 'processing', 'human', 'correction']
['applications', 'tagger', 'highest', 'possibl

In [25]:
q_lis = []
for q in queries:
    a = preprocess(q)
    if len(a)>1:
        q_lis.append(a)

['sentences', 'dso', 'collection', 'tagged', 'parts', 'speech', 'using', 'tnt', 'brants', 'trained', 'brown', 'corpus']
[]
['english', 'carried', 'using', 'freely', 'available', 'tnt', 'tagger', 'brants']
['proposition', 'quite', 'viable', 'statistical', 'pos', 'taggers', 'like', 'tnt', 'brants', 'available']
['use', 'tnt', 'brants', 'second', 'order', 'markov', 'model', 'tagger']
['pos', 'tagging', 'lemmatization', 'combine', 'genia', 'occasionally', 'deviant', 'kenizer', 'tnt', 'brants', 'operates', 'inputs', 'default', 'models', 'trained', 'financial', 'news', 'penn', 'tree', 'bank']
['tag', 'tokens', 'pos', 'tags', 'using', 'tagger', 'brants']
['example', 'petrov', 'et', 'al', 'build', 'supervised', 'pos', 'taggers', 'languages', 'using', 'tnt', 'tagger', 'brants', 'average', 'accuracy']
['forun', 'aligned', 'words', 'simply', 'assign', 'random', 'pos', 'low', 'probability', 'substantially', 'affect', 'transition', 'probability', 'estimates', 'step', 'build', 'tagger', 'feeding', '

In [26]:
q_lis

['sentences dso collection tagged parts speech using tnt brants trained brown corpus',
 'english carried using freely available tnt tagger brants',
 'proposition quite viable statistical pos taggers like tnt brants available',
 'use tnt brants second order markov model tagger',
 'pos tagging lemmatization combine genia occasionally deviant kenizer tnt brants operates inputs default models trained financial news penn tree bank',
 'tag tokens pos tags using tagger brants',
 'example petrov et al build supervised pos taggers languages using tnt tagger brants average accuracy',
 'forun aligned words simply assign random pos low probability substantially affect transition probability estimates step build tagger feeding es ti mated emission transition probabilities tnt tagger brants implementation trigram hmm tagger',
 'based various complexity grammar model using tags brants achieved automated tagging set grammatical function tags including modifiers trained supervised mode tree bank german

In [27]:
def get_matching_sentences(model):
    # Get a vector for each headline (sentence) in the corpus
    corpus_embeddings = model.encode(corpus)
    corpus_embeddings = corpus_embeddings/np.linalg.norm(corpus_embeddings,axis=0).reshape(-1)
    # Define search queries and embed them to vectors as well

    query_embeddings = model.encode(queries)
    # For each search term return 5 closest sentences
    closest_n = 5

    for i in range(len(queries)):
        query, query_embedding  = queries[i], query_embeddings[i]
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
    #     results = dict(enumerate(distances,1))
    #     results = dict(sorted(results.items(), key=lambda item: item[1]))

        print("\n\n======================\n\n")
        print("Query:", query)
        print("\nTop 5 most similar sentences in corpus:")
        indexes = results[0:closest_n]
        top_n = []
        for l,k in indexes:
            top_n.append(l)
        print(np.intersect1d(top_n,gt[cite_no[i]]))
        print("indexes{}, top_n{}, i{}, cite_no[i]{} ,gt[cite_no[i]]{}".format(indexes, top_n,i,cite_no[i],gt[cite_no[i]]))

    #     for idx, distance in results[0:closest_n]:
    #         print(corpus[idx].strip(), "(Score: %.4f)" % (1-distance))


In [28]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings:
tensor([[-0.3931,  0.0389,  1.9874,  ..., -0.6094, -1.0946,  0.3265],
        [ 0.0615,  0.3274,  1.8332,  ..., -0.1299,  0.4609,  0.2404]])


In [29]:
# Get a vector for each headline (sentence) in the corpus
corpus_embeddings = model.encode(new_corpus)
print(len(corpus_embeddings[1]))
corpus_embeddings = corpus_embeddings/np.linalg.norm(corpus_embeddings,axis=0).reshape(-1)
print(corpus_embeddings[1][0])
# Define search queries and embed them to vectors as well

query_embeddings = model.encode(q_lis)
query_embeddings  = query_embeddings/np.linalg.norm(query_embeddings ,axis=0).reshape(-1)

# For each search term return 5 closest sentences
closest_n = 10

for i in range(len(queries)):
    query, query_embedding  = queries[i], query_embeddings[i]
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
#     results = dict(enumerate(distances,1))
#     results = dict(sorted(results.items(), key=lambda item: item[1]))

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")
    indexes = results[0:closest_n]
    top_n = []
    for l,k in indexes:
        top_n.append(l)
    print(np.intersect1d(top_n,gt[cite_no[i]]))
    print("indexes{}, top_n{}, i{}, cite_no[i]{} ,gt[cite_no[i]]{}".format(indexes, top_n,i,cite_no[i],gt[cite_no[i]]))

AttributeError: 'BertModel' object has no attribute 'encode'

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# #Define your train examples. You need more than just two examples...

# train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#     InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

# #Define your train dataset, the dataloader and the train loss
# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.CosineSimilarityLoss(model)

# #Tune the model
# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=100)

In [ ]:
data = []
for i in range(len(queries)):
    for j in gt[cite_no[i]]:
        if j < len(corpus):
#             print(queries[i],gt[cite_no[i]],corpus[j])
            data.append(InputExample(texts=[queries[i],corpus[j]],label=0.8))
    
        

In [ ]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.CosineSimilarityLoss(model)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10000, warmup_steps=100)

In [ ]:
get_matching_sentences(model)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-distilroberta-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-distilroberta-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)